In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
data = pd.read_excel('dataset.xlsx')

In [ ]:
pd.set_option('display.max_row', 111)
pd.set_option('display.max_column', 111)

In [ ]:
data.head()

# 1. Exploratory Data Analysis

## Objectif :
- Comprendre du mieux possible nos données (un petit pas en avant vaut mieux qu'un grand pas en arriere)
- Développer une premiere stratégie de modélisation 

## Checklist de base
#### Analyse de Forme :
- **variable target** : SARS-Cov-2 exam result
- **lignes et colonnes** : 5644, 111
- **types de variables** : qualitatives : 37, quantitatives : 74
- **Analyse des valeurs manquantes** :
    - beaucoup de NaN (moitié des variables > 90% de NaN)
    - 2 groupes de données 76% -> Test viral, 89% -> taux sanguins
   

#### Analyse de Fond :
- **Visualisation de la target** :
    - 10 % de positifs (558 / 5000)
    

- **Signification des variables** :
    - variables continues standardisées, skewed(asymétriques), test sanguin
    - age quantile : difficile d'interpréter ce graphique, clairement ces données ont été traitées, on pourrait penser 0-5, mais cela pourrait aussi être une transformation mathématique. On ne peut pas savoir car la personne quii a mit ce dataset ne le précise nul part. Mais cela n'est pas très important.       
    - variable qualitative : binaire(0, 1), viral, Rhinovirus qui semble très élevée          


- **Relation Variables / Target** :
    - target / blood : les taux de Monocytes, Platelets, Leukocytes semblent liés au covid 19 -> hypothèse à tester
    - target /age : les indivus de faible age sont très peu contaminés ? -> attention on ne connait pas l'âge et on sait pas de quand date le dataset (s'il s'agit des enfants on sait que les enfants sont touchés autant que les adultes). En revanche cette variable pourra être intéressante pour la comparer avec les résultats de test sanguins
    - target / viral : les doubles maladies sont tres rares. Rhinovirus/Enterovirus positif - covid-19 négatif ? -> hypothese a tester ? mais il est possible que la région est subie une épidémie de ce virus. De plus on peut tres bien avoir 2 virus en meme temps. Tout ca n'a aucun lien avec le covid 19.

- **Conclusions initiales** :
    - Beaucoup de données manquantes (au mieux on garde 20 % du dataset)
    - 2 groupes de données intéressantes (viral, sanguin)
    - Presque pas de variable "discriminante" pour distinguer les cas positifs/négatifs, ce qui nous permet de dire qu'il n'est pas vraiment approprié de vouloir prédire si un individu est atteint du Covid-19 en se basant sur ces simples tests sanguins.
    Mais c'est pas grave, il faut quand même poursuivre l'analyse pour essayer de voir ce qu'on peut apprendre.
    - On a pu identifier des variables intéressantes qui sont susceptibles de jouer un role non négligeable 


    
    
    
### Analyse plus détaillée

- **Relation Variables / Variables** :
    - blood_data / blood_data : certaines variables sont très corrélées : + 0.9 (a surveiller plus tard)
    - blood_data / age : tres faible corrélation entre age et taux sanguins
    - viral / viral :influenza rapid test donne de mauvais résultat, il faudra peut-être la laisser tomber
    - relation_maladie / blood_data : les taux sanguins entre malades et covid-19 sont différents 
    - relation hospitilisation / est malade :
    - relation hospitilisation / blood : intéressant dans le cas où on voudrait prédire dans quelle service une patient devrait aller




- **NaN analyse** : 

### hypotheses nulle (H0): 
    - Les individus atteints du covid-10 ont des taux de Leukocytes, Monocytes, Platelets significativement différents
    - Les individus atteints d'une quelconque maladie ont des taux significativement différents



    

### Analyse de la forme des données

In [ ]:
df = data.copy()

In [ ]:
df.shape

In [ ]:
df.dtypes.value_counts()

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.isna(), cbar=False)

In [ ]:
(df.isna().sum()/df.shape[0]).sort_values(ascending=True)


### Analyse du fond

### 1. Visulation initiale - Elimination des colonnes inutiles

In [ ]:
# deleted_columns = df.loc[:, df.isna().sum()/df.shape[0] > 0.9]
# df.drop(deleted_columns, axis=1, inplace=True)
# df.shape

In [ ]:
df = df[df.columns[df.isna().sum()/df.shape[0] < 0.9]]
df


In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.isna(), cbar=False)

In [ ]:
df.drop(['Patient ID'], axis=1, inplace=True)

In [ ]:
df.head()

## Examen de la colonne target

In [ ]:
df['SARS-Cov-2 exam result'].value_counts(normalize=True)

### Histogramme des variables continues

In [ ]:
for col in df.select_dtypes('float'):
    plt.figure()
    sns.distplot(df[col])

In [ ]:
sns.distplot(df['Patient age quantile'])


## Variable age

In [ ]:
df['Patient age quantile'].value_counts()

## Variables Qualitatives

In [ ]:
for col in df.select_dtypes('object'):
    print(f'{col :-<50} {df[col].unique()}')

In [ ]:
for col in df.select_dtypes('object') :
    plt.figure()
    df[col].value_counts().plot.pie()

## Relation Target / Variables
### Création de sous-ensembles positifs et négatifs

In [ ]:
positive_df = df[df['SARS-Cov-2 exam result'] == 'positive']

In [ ]:
negative_df = df[df['SARS-Cov-2 exam result'] == 'negative']

### Création des ensembles Blood et viral

In [ ]:
missing_rate = df.isna().sum()/df.shape[0]

In [ ]:
blood_columns = df.columns[(missing_rate < 0.9) & (missing_rate > 0.88)]

In [ ]:
viral_columns = df.columns[(missing_rate < 0.77) & (missing_rate > 0.75)]

### Target / Blood

In [ ]:
for col in blood_columns:
    plt.figure()
    sns.distplot(positive_df[col], label='positive')
    sns.distplot(negative_df[col], label='negative')
    plt.legend()

### Target / age

### Target/ Viral

In [ ]:
for col in viral_columns:
    plt.figure()
    sns.heatmap(pd.crosstab(df['SARS-Cov-2 exam result'], df[col]), annot=True, fmt='d')

## Analyse un peu plus avancée

### Relation Variables / Variables

#### relations Taux sanguin


In [ ]:
sns.pairplot(df[blood_columns])

In [ ]:
sns.heatmap(df[blood_columns].corr())

In [ ]:
sns.clustermap(df[blood_columns].corr())

#### relation sang / age

In [ ]:
for col in blood_columns :
    plt.figure()
    sns.lmplot(x='Patient age quantile', y=col, hue='SARS-Cov-2 exam result', data=df)

In [ ]:
df.corr()['Patient age quantile'].sort_values(ascending=True)

relation viral / sanguin

#### création d'une nouvelle variable "est malade"

In [ ]:
df['est_malade'] = np.sum(df[viral_columns[:-2]] == 'detected', axis=1) >=1

In [ ]:
malade_df = df[df['est_malade'] == True]
non_malade_df = df[df['est_malade'] == False]

In [ ]:
for col in blood_columns:
    plt.figure()
    sns.distplot(malade_df[col], label="est malade")
    sns.distplot(non_malade_df[col], label="non malade")
    plt.legend()
    plt.show()

In [ ]:
def hospitalisation(df):
    if df['Patient addmited to regular ward (1=yes, 0=no)'] == 1:
        return 'surveillance'
    elif df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 1:
        return 'soins semi-intensives'
    elif df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 1:
        return 'soins intensifs'
    else:
        return 'inconnu'

In [ ]:
df['statut'] = df.apply(hospitalisation, axis=1)

### Relation hospitilisation / blood

In [ ]:
for col in blood_columns:
    plt.figure()
    for cat in df['statut'].unique():
        sns.distplot(df[df['statut']==cat][col], label=cat)
    plt.legend()